In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import PhaseEstimation, TwoLocal
from qiskit.circuit.library import HamiltonianGate
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

### import nature library
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

## custom library imports
import os
import sys
import pyscf
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
import time
import inspect
from pprint import pprint
import numpy as np

In [13]:
"""H = SparsePauliOp.from_list([('II', -1), ('IZ', 0.3), ('XI', -0.3), ('ZY', -0.01), ('YX', 0.1)])

"""
# Define the molecule
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Run the driver to get the problem
problem = driver.run()

# Define the log output (here using stdout)
flog = sys.stdout

### Apply freezecore transformer
transformer = FreezeCoreTransformer()
problem = transformer.transform(problem)

#hamiltonian = problem.hamiltonian.second_q_op()
hamiltonian_op, aux_ops = problem.second_q_ops()
#Hmat = H.to_matrix()
#print("\n".join(str(hamiltonian).splitlines()))#[:10] + ["..."]))
#print("\n".join(str(hamiltonian_op).splitlines()))#[:10] + ["..."]))
print(hamiltonian_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.2563390730032498 * ( +_0 -_0 )
+ -0.47189600728114145 * ( +_1 -_1 )
+ -1.2563390730032498 * ( +_2 -_2 )
+ -0.47189600728114145 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.33229086512764805 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.33229086512764805 * ( +_0 +_3 -_3 -_0 )
+ 0.09046559989211567 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211567 * ( +_0 +_1 -_0 -_1 )
+ 0.09046559989211567 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211567 * ( +_0 +_3 -_2 -_1 )
+ 0.09046559989211567 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211567 * ( +_1 +_1 -_0 -_0 )
+ 0.09046559989211567 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211567 * ( +_1 +_3 -_2 -_0 )
+ 0.33229086512764805 * ( +_1 +_0 -_0 -_1 )
+ 0.3492868613660088 * ( +_1 +_1 -_1 -_1 )
+ 0.33229086512764805 * ( +_1 +_2 -_2 -_1 )
+ 0.3492868613660088 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.33229086512764805 * ( +_2 +_1 -_1 -_2 )
+ 0.3

In [3]:
eig = np.linalg.eigvals(hamiltonian)
print('NUMPY eig',eig)

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')

vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)
print('VQE eig', result.eigenvalue)

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [ ]:

n_qpe_qbits = 10

U = HamiltonianGate(Hmat, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)
Qpe=qpe.decompose().decompose().decompose().decompose().decompose().decompose().decompose()
measure_circ = measure_circ.compose(Qpe)

measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))
#measure_circ.draw("mpl")


In [ ]:
from qiskit_aer import Aer
backend=Aer.get_backend('aer_simulator')
job=backend.run(measure_circ,shots=10)
result=job.result()
counts = job.result().get_counts()
print(counts)

In [ ]:
max_count = max(counts.items(), key=lambda x: x[1])
print(f'MAX count: {max_count}')

theta = int(max_count[0][::-1], 2) / 2**n_qpe_qbits
print(f'Theta value: {theta}')
print(f'QPE-approximated U-eigenvalue: {np.exp(2*1j*np.pi * theta)}')
print(f'QPE-approximated H-eigenvalue: {-2 * np.pi * theta}')